In [9]:
#######
# cmd #
#######
# ! python -m pip install --upgrade pip
# ! pip install --pre pandas==2.0.0rc0

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import warnings

pd.options.mode.dtype_backend = 'pyarrow'

############################################
########## IMPORTACAO DOS DADOS ############
############################################
# 1m 15.9s w/ old pandas
# 5.9s w/ new pandas

INPUT_DIR = 'C:/Users/u00378/Desktop/PIBIC_2021-2022/data_from_kaggle'
#INPUT_DIR = 'C:/Users/Igor/Desktop/PIBIC/PIBIC_2021-2022/data_from_kaggle'

calendar = pd.read_csv(f'{INPUT_DIR}/calendar.csv', engine='pyarrow', use_nullable_dtypes=True)
selling_prices = pd.read_csv(f'{INPUT_DIR}/sell_prices.csv', engine='pyarrow', use_nullable_dtypes=True)
sample_submission = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv', engine='pyarrow', use_nullable_dtypes=True)
sales_train_val = pd.read_csv(f'{INPUT_DIR}/sales_train_validation.csv', engine='pyarrow', use_nullable_dtypes=True) # sales_train d_1 ate
                                                                                                                            # d_1913
sales_train_eva = pd.read_csv(f'{INPUT_DIR}/sales_train_evaluation.csv',engine='pyarrow', use_nullable_dtypes=True) # sales_train d_1 ate
                                                                                                                            # d_1941
cols = []

for i in range(1, 1942, 1):
    col = f"d_{i}"
    cols.append(col)

sales_train = pd.concat([sales_train_val, sales_train_eva[cols[-28:]]], axis = 1)
sales_train['id'] = sales_train['id'].apply(lambda w : w.replace('_validation', ''))

############
### Note ###
############
# sales_train_eva contempla o sales_train_val e ainda adciona observacoes das vendas dos dias d_1914 - d_1941
# id = ..._validation => até d_1913
# id = ..._evaluation => até d_1941

############
### Goal ###
############
# validation part of submission sample => cross validation w/ d_1 to d_1913 => calculate sMAPE e MASE w/ d_1914 to d_1941
# evaluation part of submission sample => cross validation w/ d_1 to d_1941 => calculate M5 final score in kaggle by
                                                                                    # concatenating these parts

#################################################
### Filtros para alcançar as series temporais ###
#################################################
def filter_item_store(item_id, store_id):
    '''
    Funcao para filtrar os dados no nivel mais desagregado possivel - venda do item_id na store_id

    '''
    v = sales_train[(sales_train['item_id']== item_id) & (sales_train['store_id']== store_id)][cols].sum().values

    return pd.Series(v, index = calendar['date'][:-28], dtype = 'int64[pyarrow]')


def filter_dept_store(dept_id, store_id):
    '''
    Funcao para filtrar os dados no nivel da venda de todos os item_id do dept_id na store_id

    '''
    v = sales_train[(sales_train['dept_id']== dept_id) & (sales_train['store_id']== store_id)][cols].sum().values

    return pd.Series(v, index = calendar['date'][:-28], dtype = 'int64[pyarrow]')

def filter_store(store_id):
    '''
    Funcao para filtrar os dados no nivel da venda de todos os item_id na store_id

    '''
    v = sales_train[sales_train['store_id'] == store_id][cols].sum().values

    return pd.Series(v, index = calendar['date'][:-28], dtype = 'int64[pyarrow]')


def no_filter():
    '''
    Funcao para alcancar as vendas no nivel mais agregado possivel - vendas diarias da walmart como um todo

    '''
    v = sales_train[cols].sum().values

    return pd.Series(v, index = calendar['date'][:-28], dtype = 'int64[pyarrow]')

## DEFININDO O FILTRO PARA BUSCAR AS EXPLANATORY VARIABLES

In [69]:
def get_price(item_id, store_id):
    '''
    retorna o preço do item_id vendido na store_id na semana wm_yr_wk

    '''
    v = selling_prices[(selling_prices['store_id'] == store_id) & (selling_prices['item_id'] == item_id)][['wm_yr_wk', 'sell_price']]

    return v

In [71]:
get_price('HOBBIES_1_001', 'CA_1')

,wm_yr_wk,sell_price
0,11325,9.58
1,11326,9.58
2,11327,8.26
3,11328,8.26
4,11329,8.26
...,...,...
149,11617,8.38
150,11618,8.38
151,11619,8.38
152,11620,8.38


In [29]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,,,,,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,,,,,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,,,,,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,,,,,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,,,,,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,,,,,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,,,,,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,,,,,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,,,,,0,0,0


In [43]:
calendar[['date', 'd', 'wday', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI']]

,date,d,wday,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,d_1,1,,,,,0,0,0
1,2011-01-30,d_2,2,,,,,0,0,0
2,2011-01-31,d_3,3,,,,,0,0,0
3,2011-02-01,d_4,4,,,,,1,1,0
4,2011-02-02,d_5,5,,,,,1,0,1
...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,d_1965,5,,,,,0,1,1
1965,2016-06-16,d_1966,6,,,,,0,0,0
1966,2016-06-17,d_1967,7,,,,,0,0,0
1967,2016-06-18,d_1968,1,,,,,0,0,0


In [26]:
np.unique(calendar['event_type_1'])

array(['', 'Cultural', 'National', 'Religious', 'Sporting'], dtype=object)

In [27]:
np.unique(calendar['event_type_2'])

array(['', 'Cultural', 'Religious'], dtype=object)